<a href="https://colab.research.google.com/github/7mansi/kaggle/blob/playground-series/kagglePractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle


In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mansimanno","key":"9e56e686ba3e283e0456e3d62d0d3654"}'}

In [4]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
 ! kaggle datasets list

ref                                                        title                                           size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------------  ---------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
salvatorerastelli/spotify-and-youtube                      Spotify and Youtube                              9MB  2023-03-20 15:43:25           5668        215  1.0              
erdemtaha/cancer-data                                      Cancer Data                                     49KB  2023-03-22 07:57:00           1529         40  1.0              
ulrikthygepedersen/fastfood-nutrition                      Fastfood Nutrition                              12KB  2023-03-21 10:02:41           1751         40  1.0              
lokeshparab/amazon-products-dataset                        Amazon Products Sales Dataset 2023              80M

In [7]:
!kaggle competitions download -c 'playground-series-s3e12'

  0% 0.00/9.06k [00:00<?, ?B/s]
100% 9.06k/9.06k [00:00<00:00, 13.2MB/s]


In [8]:
! unzip playground-series-s3e12.zip -d playground-series-s3e12

Archive:  playground-series-s3e12.zip
  inflating: playground-series-s3e12/sample_submission.csv  
  inflating: playground-series-s3e12/test.csv  
  inflating: playground-series-s3e12/train.csv  


In [9]:
! unzip playground-series-s3e12.zip

Archive:  playground-series-s3e12.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [32]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import  MinMaxScaler,StandardScaler
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

In [46]:
scaling= StandardScaler()
import pandas as pd
data= pd.read_csv("train.csv")
data


     gravity    ph  osmo  cond  urea  calc
0      1.013  6.19   443  14.8   124  1.45
1      1.025  5.40   703  23.6   394  4.18
2      1.009  6.13   371  24.5   159  9.04
3      1.021  4.91   442  20.8   398  6.63
4      1.021  5.53   874  17.8   385  2.21
..       ...   ...   ...   ...   ...   ...
409    1.011  5.21   527  21.4    75  1.53
410    1.024  5.53   577  19.7   224  0.77
411    1.018  6.28   455  22.2   270  7.68
412    1.008  7.12   325  12.6    75  1.03
413    1.011  6.13   364   9.9   159  0.27

[414 rows x 6 columns]


In [63]:
data_scaled= scaling.fit_transform(data)

In [64]:
print(data_scaled)
#plt.scatter(X_scaled['gravity'],data.target, c='g')

[[-1.72787215 -0.73403157  0.36562268 ... -1.13486936 -0.82913603
  -0.89442719]
 [-1.71950473  1.06590369 -0.86589692 ...  0.84638411  0.02033832
  -0.89442719]
 [-1.71113731 -1.33400999  0.27208954 ... -0.8780402   1.53258937
  -0.89442719]
 ...
 [ 1.71113731  0.01594146  0.50592238 ... -0.06352489  1.109408
   1.11803399]
 [ 1.71950473 -1.48400459  1.81538625 ... -1.49443017 -0.9598244
   1.11803399]
 [ 1.72787215 -1.03402078  0.27208954 ... -0.8780402  -1.1963081
  -0.89442719]]


In [66]:
data['gravity'].describe()

count    414.000000
mean       1.017894
std        0.006675
min        1.005000
25%        1.012000
50%        1.018000
75%        1.022000
max        1.040000
Name: gravity, dtype: float64

In [67]:
data.describe()

,id,gravity,ph,osmo,cond,urea,calc,target
count,414.000000,414.000000,414.000000,414.000000,414.000000,414.000000,414.000000,414.000000
mean,206.500000,1.017894,5.955459,651.545894,21.437923,278.657005,4.114638,0.444444
std,119.655756,0.006675,0.642260,234.676567,7.514750,136.442249,3.217641,0.497505
min,0.000000,1.005000,4.760000,187.000000,5.100000,10.000000,0.170000,0.000000
25%,103.250000,1.012000,5.530000,455.250000,15.500000,170.000000,1.450000,0.000000
50%,206.500000,1.018000,5.740000,679.500000,22.200000,277.000000,3.130000,0.000000
75%,309.750000,1.022000,6.280000,840.000000,27.600000,385.000000,6.630000,1.000000
max,413.000000,1.040000,7.940000,1236.000000,38.000000,620.000000,14.340000,1.000000


In [98]:
data= data[~(data['urea']>525)]
data=data[~(data['urea']<50)]
print(data)



data_scaled= scaling.fit_transform(data)
print(data_scaled)
X= data_scaled[:, :5]
Y= data_scaled[:,6]


     gravity    ph  osmo  cond  urea  calc  target
0      1.013  6.19   443  14.8   124  1.45       0
1      1.025  5.40   703  23.6   394  4.18       0
2      1.009  6.13   371  24.5   159  9.04       0
3      1.021  4.91   442  20.8   398  6.63       1
4      1.021  5.53   874  17.8   385  2.21       1
..       ...   ...   ...   ...   ...   ...     ...
409    1.011  5.21   527  21.4    75  1.53       0
410    1.024  5.53   577  19.7   224  0.77       0
411    1.018  6.28   455  22.2   270  7.68       1
412    1.008  7.12   325  12.6    75  1.03       1
413    1.011  6.13   364   9.9   159  0.27       0

[387 rows x 7 columns]
[[-0.69740762  0.34786487 -0.87367978 ... -1.17410745 -0.81208123
  -0.88975652]
 [ 1.16394927 -0.88392486  0.35806549 ...  1.13308192  0.07534437
  -0.88975652]
 [-1.31785991  0.25431121 -1.21477847 ... -0.87502734  1.65515697
  -0.88975652]
 ...
 [ 0.07815775  0.48819534 -0.81683    ...  0.07348384  1.21306949
   1.12390297]
 [-1.47297299  1.79794645 -1.432702